In [43]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

In [44]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
import torch
import numpy as np
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'
work_folder = '../local_work'

## Calculate Standard Deviation

In [47]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-resnet1d-stainless-2022-07-11_01-00.ckpt") #CHANGEME inconel vs stainless best model
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-resnet1d-stainless-2022-07-11_01-00.ckpt


In [48]:
from src.models import InverseModel

i_model = InverseModel.load_from_checkpoint(filepath, strict=False)
i_model.eval()

x = torch.from_numpy(np.array(torch.randn(1, 800) * 3)) #
with torch.no_grad():
    y_hat = i_model(x)
y_hat.shape

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

### Unexplored Params

In [86]:
inc_data = torch.load(
                Path(f"{data_folder}/inconel-revised-raw-shuffled.pt"))["laser_params"]
params = torch.load(
                Path(f"{data_folder}/stainless-steel-revised-shuffled.pt"))["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
max_speed, max_spacing = params.max(0)[0][0].item(), params.max(0)[0][1].item()
min_speed, min_spacing = params.min(0)[0][0].item(), params.min(0)[0][1].item()
print(max_speed, max_spacing)
print(min_speed, min_spacing)

700.0 42.0
10.0 1.0


In [87]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = max_speed * y_hat[0][0] # TODO call the scale
    y_final[1] = max_spacing * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    return y_final

In [88]:
#identify explored param space
def get_unexplored_params(params, data, max_speed, max_spacing, min_speed, min_spacing, n = 196, seed = 1):
        """
        finds unexplored parameters
        inputs: tensor of [[], [], []], each size (1,14)
        outputs: list of n unnormalized tuples each of size (1,3)
        """
        param_space = [np.arange(min_speed, max_speed + 10, 10).tolist(), np.arange(min_spacing, max_spacing + 1, 1).tolist(), np.around(np.arange(.2, 1.4, .1), decimals = 1).tolist()]
        #get set of all possible parameter combinations
        param_all_comb = set(list(itertools.product(*param_space)))#[list(tup) for tup in itertools.product(*param_space)]
        wattages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
        #get set of all training parameter combinations -decoding of one hot encoding
        train_param_space = set([tuple(x[:2] + [wattages[np.argmax(x[2:])]]) for x in data.tolist()]) #29988
        #find the combination parameters NOT in the training space
        unused_params = list(param_all_comb.symmetric_difference(train_param_space)) #len() = 5292
        #sample n from unused parameter combinations
        random.seed = seed
        indices = random.sample(range(len(unused_params)), n)
        return [unused_params[i] for i in indices]

def unexplored_params_to_tensors(x):
        new_x = []
        wattage_idxs = {
                0.2: 0,
                0.3: 1,
                0.4: 2,
                0.5: 3,
                0.6: 4,
                0.7: 5,
                0.8: 6,
                0.9: 7,
                1.0: 8,
                1.1: 9,
                1.2: 10,
                1.3: 11,
        }      
        for i in x:
                i = list(i)
                holder = i[:2] + np.zeros(12).tolist()
                holder[wattage_idxs[i[2]] + 2] = 1.0
                new_x.append(holder)

        laser_params = torch.FloatTensor(new_x)
        for r in laser_params:
                r[0]=r[0] / max_speed
                r[1]=r[1] / max_spacing
        return laser_params

#[torch.cat((x[:2], torch.tensor(np.array([wattages[np.argmax(x[2:])]])))) for x in params]#[:5]
#[x[:2] for x in params]#[:5]

# train_param_space = denormalize_decode_result(params, max_speed, max_spacing, min_speed, min_spacing)
# train_param_space


In [89]:
ouput_unexplored = get_unexplored_params(params, inc_data, max_speed, max_spacing, min_speed, min_spacing, n = 1200, seed = 1)
norm_unexplored_laser_params = unexplored_params_to_tensors(ouput_unexplored)

In [90]:
norm_unexplored_laser_params

tensor([[0.6857, 0.7381, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.1000, 0.2619, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.2000, 1.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        ...,
        [0.7714, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.1857, 0.7381, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.7571, 0.2381, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

# Inverse Model Only


In [5]:
from src.models import InverseModel

In [6]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-resnet1d-stainless-2022-07-11_01-00") #CHANGEME inconel vs stainless best model
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt


In [7]:
i_model = InverseModel.load_from_checkpoint(filepath, strict=False)
i_model.eval()

x = torch.from_numpy(np.array(torch.randn(1, 800) * 3)) #
with torch.no_grad():
    y_hat = i_model(x)
y_hat.shape

KeyError: 'inconel'

In [94]:
# y_final2 = onehot_to_continuous2(y_hat)
print(denormalize_decode_result(y_hat,max_speed, max_spacing, min_speed, min_spacing))

#if you want hardcoded max, min values instead
#print(denormalize_decode_result(y_hat,700.0, 42.0, 10.0, 1.0))

tensor([ 4.7735, 36.7123,  1.1000])


# Direct Model Only


In [95]:
from src.models import DirectModel

In [96]:
filepath = Path(
    f"{work_folder}/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt")
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt


In [97]:
d_model = DirectModel.load_from_checkpoint(filepath)
d_model.eval()
# x = torch.randn(1, 14)
direct_y_hats = []
# intersect_preds = []
for i in norm_unexplored_laser_params:
    with torch.no_grad():
        direct_y_hats.append(d_model(i.reshape(1,14)))

### Unexplored Direct --> Indirect

In [98]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt") #CHANGEME inconel vs stainless best model
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt


In [99]:
model = InverseModel.load_from_checkpoint(filepath, strict=False)
model.eval()

#x = torch.from_numpy(np.array(torch.ra|ndn(1, 800) * 3)) #
param_preds = []
for i in direct_y_hats:
    with torch.no_grad():
        param_preds.append(model(i))#model(i))

In [100]:
params = torch.load(
                Path(f"{data_folder}/inconel-revised-raw-shuffled.pt"))["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
max_speed, max_spacing = params.max(0)[0][0].item(), params.max(0)[0][1].item()
min_speed, min_spacing = params.min(0)[0][0].item(), params.min(0)[0][1].item()
decoded_preds = [denormalize_decode_result(pred, max_speed, max_spacing, min_speed, min_spacing) for pred in param_preds]
for r in decoded_preds:
    r[0]=torch.round(r[0], decimals = -1)
    r[1]=torch.round(r[1], decimals = 0)

In [101]:
param_space = [np.arange(min_speed, max_speed + 10, 10).tolist(), np.arange(min_spacing, max_spacing + 1, 1).tolist(), np.around(np.arange(.2, 1.4, .1), decimals = 1).tolist()]
#get set of all possible parameter combinations
param_all_comb = set(list(itertools.product(*param_space)))
wattages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
space_params = set([tuple(np.around(x.tolist(), decimals= 1)) for x in decoded_preds]) #29988
        #find the combination parameters NOT in the training space
overlaps = list(param_all_comb.intersection(space_params)) #len() = 5292
unused_params = list(set(overlaps).symmetric_difference(space_params)) #len() = 5292
#convert preds to set
print(len(space_params))
print(len(overlaps)) #332
len(unused_params) #216



591
352


239

In [63]:
#send these to minok
count_samples_to_minok = 196
unused_params[:count_samples_to_minok]#overlaps)
#overlaps[:195]


[(570.0, 13.0, 1.2),
 (330.0, 13.0, 1.2),
 (610.0, 14.0, 1.2),
 (340.0, 10.0, 1.2),
 (550.0, 11.0, 1.2),
 (510.0, 9.0, 1.2),
 (500.0, 7.0, 1.2),
 (660.0, 9.0, 1.2),
 (370.0, 12.0, 1.2),
 (440.0, 9.0, 1.2),
 (650.0, 10.0, 1.2),
 (490.0, 13.0, 1.2),
 (220.0, 12.0, 1.2),
 (610.0, 13.0, 1.2),
 (420.0, 7.0, 1.2),
 (180.0, 10.0, 1.2),
 (640.0, 8.0, 1.2),
 (470.0, 11.0, 1.2),
 (460.0, 8.0, 1.2),
 (340.0, 13.0, 1.2),
 (510.0, 12.0, 1.2),
 (580.0, 9.0, 1.2),
 (100.0, 10.0, 1.2),
 (280.0, 9.0, 1.2),
 (260.0, 11.0, 1.2),
 (450.0, 14.0, 1.2),
 (630.0, 13.0, 1.2),
 (440.0, 12.0, 1.2),
 (140.0, 12.0, 1.2),
 (420.0, 10.0, 1.2),
 (360.0, 10.0, 1.2),
 (670.0, 11.0, 0.5),
 (170.0, 8.0, 1.2),
 (100.0, 13.0, 1.2),
 (380.0, 9.0, 1.2),
 (520.0, 12.0, 1.2),
 (500.0, 10.0, 1.2),
 (160.0, 14.0, 1.2),
 (200.0, 10.0, 1.2),
 (550.0, 14.0, 1.2),
 (620.0, 11.0, 1.2),
 (130.0, 9.0, 1.2),
 (360.0, 13.0, 1.2),
 (480.0, 13.0, 1.2),
 (640.0, 11.0, 1.2),
 (0.0, 28.0, 0.8),
 (180.0, 13.0, 1.2),
 (600.0, 9.0, 1.2),
 (570.0

In [256]:
#intersecting parameters!!!!!!!
intersect_x = []
wattage_idxs = {
        0.2: 0,
        0.3: 1,
        0.4: 2,
        0.5: 3,
        0.6: 4,
        0.7: 5,
        0.8: 6,
        0.9: 7,
        1.0: 8,
        1.1: 9,
        1.2: 10,
        1.3: 11,
}      
for i in unused_params:
        i = list(i)
        holder = i[:2] + np.zeros(12).tolist()
        holder[wattage_idxs[i[2]] + 2] = 1.0
        intersect_x.append(holder)
output = torch.tensor(intersect_x)
output

tensor([[600.,  34.,   0.,  ...,   0.,   0.,   0.],
        [590.,  14.,   0.,  ...,   0.,   1.,   0.],
        [580.,  31.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [640.,  16.,   0.,  ...,   0.,   1.,   0.],
        [520.,  38.,   0.,  ...,   0.,   0.,   0.],
        [300.,  37.,   0.,  ...,   0.,   0.,   0.]], dtype=torch.float64)

In [136]:
for i in range(len(params)):
    curr = params[i].flatten().tolist()
    if curr in output.tolist():
        print(i)

12058
19562
19609


In [149]:
from utils import rmse
# params[12058]
data = torch.load(
                Path(f"{data_folder}/stainless-steel-revised-shuffled.pt"))

intersect_train_emiss = []
intersect_train_emiss.append(data["emissivity"][12058])
intersect_train_emiss.append(data["emissivity"][19562])
intersect_train_emiss.append(data["emissivity"][19609])

# ONNX

In [ ]:
# import onnxruntime 
# from pathlib import Path
# import numpy as np
# import os

In [ ]:
ort_session = onnxruntime.InferenceSession(filepath)

In [ ]:
# filepath = Path(f"{work_folder}/saved_best/best_inference_direct.onnx")

# ort_session = onnxruntime.InferenceSession(filepath)
# input_name = ort_session.get_inputs()[0].name
# ort_inputs = {input_name: np.random.randn(2, 14)}
# ort_outs = ort_session.run(None, ort_inputs)
